In [ ]:
# | default_exp classes.DomoInstanceConfig_UserAttribute

In [ ]:
# | export
from domolibrary.routes.user_attributes import (
    UserAttributes_GET_Error,
    UserAttributes_CRUD_Error,
    UserAttributes_IssuerType,
)

In [ ]:
# | exporti

from dataclasses import dataclass, field

import httpx
from nbdev.showdoc import patch_to
from typing import List, Any

import domolibrary.client.DomoAuth as dmda
import domolibrary.routes.user_attributes as user_attribute_routes

import datetime as dt

# import asyncio
# import sys
# import pandas as pd


# from typing import List

# import domolibrary.utils.DictDot as util_dd
# import domolibrary.utils.chunk_execution as ce

# import domolibrary.client.DomoError as de
# import domolibrary.client.Logger as lg

# import domolibrary.routes.instance_config as instance_config_routes
# import domolibrary.routes.bootstrap as bootstrap_routes
# import domolibrary.routes.sandbox as sandbox_routes
# import domolibrary.routes.publish as publish_routes
# import domolibrary.routes.application as application_routes

In [ ]:
# | hide
from nbdev.showdoc import show_doc
import os

In [ ]:
# | export


@dataclass
class UserAttribute:
    """utility class that absorbs many of the domo instance configuration methods"""

    auth: dmda.DomoAuth = field(repr=False)
    id: str
    name: str
    description: str

    issuer_type: UserAttributes_IssuerType
    customer_id: str
    value_type: str

    validator: str
    validator_configuration: None

    security_voter: str
    custom: bool

    def __eq__(self, other):
        return self.id == other.id

    @classmethod
    def _from_json(cls, obj, auth):
        return cls(
            auth=auth,
            id=obj["key"],
            name=obj["title"],
            description=obj["description"],
            issuer_type=UserAttributes_IssuerType(obj["keyspace"]),
            customer_id=obj["context"],
            value_type=obj["valueType"],
            validator=obj["validator"],
            validator_configuration=obj["validatorConfiguration"],
            security_voter=obj["securityVoter"],
            custom=obj["custom"],
        )

In [ ]:
# | exporti


@patch_to(UserAttribute, cls_method=True)
async def get_by_id(
    cls: UserAttribute,
    auth: dmda.DomoAuth,
    attribute_id: str,
    session: httpx.AsyncClient = None,
    debug_api: bool = False,
    debug_num_stacks_to_drop=2,
    return_raw: bool = False,
):
    res = await user_attribute_routes.get_user_attribute_by_id(
        auth=auth,
        attribute_id=attribute_id,
        session=session,
        debug_api=debug_api,
        parent_class=cls.__name__,
        debug_num_stacks_to_drop=debug_num_stacks_to_drop,
    )

    if return_raw:
        return res
    return cls._from_json(obj=res.response, auth=auth)

In [ ]:
show_doc(UserAttribute.get_by_id)

---

[source](https://github.com/jaewilson07/domo_library/blob/main/domolibrary/classes/DomoInstanceConfig_UserAttributes.py#L87){target="_blank" style="float:right; font-size:smaller"}

### UserAttribute.get_by_id

>      UserAttribute.get_by_id (auth:domolibrary.client.DomoAuth.DomoAuth,
>                               attribute_id:str,
>                               session:httpx.AsyncClient=None,
>                               debug_api:bool=False,
>                               debug_num_stacks_to_drop=2,
>                               return_raw:bool=False)

In [ ]:
# import os
# import domolibrary.client.DomoAuth as dmda

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

await UserAttribute.get_by_id(
    attribute_id="DomoLibraryTestFranchise", auth=token_auth, debug_api=False
)

UserAttribute(id='DomoLibraryTestFranchise', name='franchise_id', description='test', issuer_type=<UserAttributes_IssuerType.CUSTOM: 'customer-defined'>, customer_id='mmmm-0012-0200', value_type='STRING', validator='ANY_VALUE', validator_configuration=None, security_voter='FULL_VIS_ADMIN_IDP', custom=True)

In [ ]:
# |exporti
@patch_to(UserAttribute)
async def update(
    self: UserAttribute,
    name=None,
    description=None,
    issuer_type: UserAttributes_IssuerType = None,
    data_type: str = None,
    security_voter=None,
    session: httpx.AsyncClient = None,
    debug_api: bool = False,
    debug_num_stacks_to_drop=2,
):
    await user_attribute_routes.update_user_attribute(
        auth=self.auth,
        attribute_id=self.id,
        name=name,
        description=description,
        issuer_type=issuer_type,
        data_type=data_type,
        security_voter=security_voter,
        session=session,
        debug_api=debug_api,
        parent_class=self.__class__.__name__,
        debug_num_stacks_to_drop=debug_num_stacks_to_drop,
    )

    new = await UserAttribute.get_by_id(attribute_id=self.id, auth=self.auth)

    [setattr(self, key, value) for key, value in new.__dict__.items()]

    return self

In [ ]:
show_doc(UserAttribute.update)

---

[source](https://github.com/jaewilson07/domo_library/blob/main/domolibrary/classes/DomoInstanceConfig_UserAttributes.py#L112){target="_blank" style="float:right; font-size:smaller"}

### UserAttribute.update

>      UserAttribute.update (name=None, description=None, issuer_type:domolibrar
>                            y.routes.user_attributes.UserAttributes_IssuerType=
>                            None, data_type:str=None, security_voter=None,
>                            session:httpx.AsyncClient=None,
>                            debug_api:bool=False, debug_num_stacks_to_drop=2)

In [ ]:
# import os
# import domolibrary.client.DomoAuth as dmda

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)
try:
    domo_user_attribute = await UserAttribute.get_by_id(
        auth=token_auth, attribute_id="TestLibraryStore"
    )

    res = await domo_user_attribute.update(
        description=f"domo_library test:  updated from classmethod, {dt.datetime.now().strftime('%Y-%m-%d - %H:%M')}"
    )
    print(res)

except (UserAttributes_GET_Error, UserAttributes_CRUD_Error) as e:
    print(e)

🛑  UserAttributes_GET_Error 🛑 - function: UserAttribute.get_user_attributes || status 200 || attribute TestLibraryStore not found at domo-community


In [ ]:
# | export


@dataclass
class UserAttributes:
    auth: dmda.DomoAuth = field(repr=False)

    attributes: List[UserAttribute] = field(default=None)

In [ ]:
# | exporti
@patch_to(UserAttributes)
async def get_attributes(
    self: UserAttributes,
    issuer_type_ls: List[
        UserAttributes_IssuerType
    ] = None,  # use `UserAttributes_IssuerType` enum
    session: httpx.AsyncClient = None,
    debug_api: bool = False,
    debug_num_stacks_to_drop=2,
):
    auth = self.auth

    res = await user_attribute_routes.get_user_attributes(
        auth=auth,
        session=session,
        issuer_type_ls=issuer_type_ls,
        debug_api=debug_api,
        parent_class=self.__class__.__name__,
        debug_num_stacks_to_drop=debug_num_stacks_to_drop,
    )

    self.attributes = [
        UserAttribute._from_json(obj=obj, auth=auth) for obj in res.response
    ]
    return self.attributes

In [ ]:
show_doc(UserAttributes.get_attributes)

---

[source](https://github.com/jaewilson07/domo_library/blob/main/domolibrary/classes/DomoInstanceConfig_UserAttribute.py#L149){target="_blank" style="float:right; font-size:smaller"}

### UserAttributes.get_attributes

>      UserAttributes.get_attributes (issuer_type_ls:List[domolibrary.routes.use
>                                     r_attributes.UserAttributes_IssuerType]=No
>                                     ne, session:httpx.AsyncClient=None,
>                                     debug_api:bool=False,
>                                     debug_num_stacks_to_drop=2)

In [ ]:
# import os
# import domolibrary.client.DomoAuth as dmda

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

domo_user_attributes = UserAttributes(auth=token_auth)

await domo_user_attributes.get_attributes(debug_api=False)

[UserAttribute(id='DomoLibraryStore', name='store_id', description='store ids', issuer_type=<UserAttributes_IssuerType.CUSTOM: 'customer-defined'>, customer_id='mmmm-0012-0200', value_type='STRING', validator='ANY_VALUE', validator_configuration=None, security_voter='FULL_VIS_ADMIN_IDP', custom=True),
 UserAttribute(id='DomoLibraryTestFranchise', name='franchise_id', description='test', issuer_type=<UserAttributes_IssuerType.CUSTOM: 'customer-defined'>, customer_id='mmmm-0012-0200', value_type='STRING', validator='ANY_VALUE', validator_configuration=None, security_voter='FULL_VIS_ADMIN_IDP', custom=True),
 UserAttribute(id='domo.policy.managed_email_address', name=None, description=None, issuer_type=<UserAttributes_IssuerType.SYSTEM: 'domo-defined'>, customer_id=None, value_type=None, validator='ANY_VALUE', validator_configuration=None, security_voter='FULL_VIS_ADMIN_IDP', custom=False),
 UserAttribute(id='domo.policy.managed_employee_id', name=None, description=None, issuer_type=<User

In [ ]:
# | exporti


@patch_to(UserAttributes)
async def create(
    self: UserAttributes,
    attribute_id: str,
    name=None,
    description=f"updated via domolibrary {dt.datetime.now().strftime('%Y-%m-%d - %H:%M')}",
    data_type: str = "ANY_VALUE",
    security_voter="FULL_VIS_ADMIN_IDP",
    issuer_type: UserAttributes_IssuerType = UserAttributes_IssuerType.CUSTOM,
    session: httpx.AsyncClient = None,
    debug_api: bool = False,
    debug_num_stacks_to_drop=2,
    return_raw: bool = False,
):

    auth = self.auth
    attribute_id = user_attribute_routes.clean_attribute_id(attribute_id)

    res = await user_attribute_routes.create_user_attribute(
        auth=auth,
        session=session,
        issuer_type=issuer_type,
        name=name,
        attribute_id=attribute_id,
        description=description,
        data_type=data_type,
        security_voter=security_voter,
        debug_api=debug_api,
        parent_class=self.__class__.__name__,
        debug_num_stacks_to_drop=debug_num_stacks_to_drop,
    )

    await self.get_attributes()

    if return_raw:
        return res

    return await UserAttribute.get_by_id(auth=auth, attribute_id=attribute_id)

In [ ]:
show_doc(UserAttributes.create)

---

[source](https://github.com/jaewilson07/domo_library/blob/main/domolibrary/classes/DomoInstanceConfig_UserAttribute.py#L176){target="_blank" style="float:right; font-size:smaller"}

### UserAttributes.create

>      UserAttributes.create (attribute_id:str, name=None, description='updated
>                             via domolibrary 2024-01-03 - 04:50',
>                             data_type:str='ANY_VALUE',
>                             security_voter='FULL_VIS_ADMIN_IDP', issuer_type:d
>                             omolibrary.routes.user_attributes.UserAttributes_I
>                             ssuerType=<UserAttributes_IssuerType.CUSTOM:
>                             'customer-defined'>,
>                             session:httpx.AsyncClient=None,
>                             debug_api:bool=False, debug_num_stacks_to_drop=2,
>                             return_raw:bool=False)

In [ ]:
# import os
# import domolibrary.client.DomoAuth as dmda

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

domo_user_attributes = UserAttributes(auth=token_auth)
try:
    res = await domo_user_attributes.create(
        attribute_id="Test Library Store", name="test library store"
    )
    print(res)

except UserAttributes_CRUD_Error as e:
    print(e)

UserAttribute(id='TestLibraryStore', name='test library store', description='updated via domolibrary 2024-01-03 - 04:50', issuer_type=<UserAttributes_IssuerType.CUSTOM: 'customer-defined'>, customer_id='mmmm-0012-0200', value_type='STRING', validator='ANY_VALUE', validator_configuration=None, security_voter='FULL_VIS_ADMIN_IDP', custom=True)


In [ ]:
# | exporti


@patch_to(UserAttributes)
async def upsert(
    self: UserAttributes,
    attribute_id,
    name=None,
    description=None,
    issuer_type: UserAttributes_IssuerType = None,
    data_type: str = None,
    security_voter=None,
    session: httpx.AsyncClient = None,
    debug_api: bool = False,
    debug_num_stacks_to_drop=2,
    debug_prn: bool = False,
):

    auth = self.auth
    attribute_id = user_attribute_routes.clean_attribute_id(attribute_id)

    user_attribute = None

    try:
        user_attribute = await UserAttribute.get_by_id(
            attribute_id=attribute_id,
            auth=auth,
            session=session,
            debug_num_stacks_to_drop=debug_num_stacks_to_drop,
            debug_api=debug_api,
        )

        if user_attribute:
            if debug_prn:
                print(f"upserting {attribute_id} in {auth.domo_instance}")

            await user_attribute.update(
                name=name,
                description=description,
                issuer_type=issuer_type,
                data_type=data_type,
                security_voter=security_voter,
                session=session,
                debug_api=debug_api,
                debug_num_stacks_to_drop=debug_num_stacks_to_drop,
            )

        return user_attribute

    except (UserAttributes_CRUD_Error, UserAttributes_GET_Error) as e:
        if debug_prn:
            print(f"creating {attribute_id} in {auth.domo_instance}")

        return await self.create(
            attribute_id=attribute_id,
            name=name,
            description=description,
            issuer_type=issuer_type or UserAttributes_IssuerType.CUSTOM,
            data_type=data_type,
            security_voter=security_voter,
            session=session,
            debug_api=debug_api,
            debug_num_stacks_to_drop=debug_num_stacks_to_drop + 1,
        )

    finally:
        await self.get_attributes()

In [ ]:
# import os
# import domolibrary.client.DomoAuth as dmda

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

domo_user_attributes = UserAttributes(auth=token_auth)

await domo_user_attributes.upsert(
    attribute_id="TestLibraryStore",
    description=f"domo_libary upsert test {dt.datetime.now().strftime('%Y-%m-%d - %H:%M')}",
    debug_prn=True,
    debug_api=True,
)

creating TestLibraryStore in domo-community
🐛 debugging get_data
{'body': {'description': 'domo_libary upsert test 2024-01-03 - 04:51',
          'key': 'TestLibraryStore',
          'keyspace': 'customer-defined',
          'securityVoter': 'FULL_VIS_ADMIN_IDP',
          'title': 'TestLibraryStore',
          'validator': 'ANY_VALUE'},
 'function_name': 'create',
 'headers': {'Connection': 'keep-alive',
             'Content-Type': 'application/json',
             'accept': 'application/json, text/plain',
             'x-domo-developer-token': '83ece44f1451d4b581e1191f98cd411164f0b5b6ad2755b3'},
 'method': 'POST',
 'params': None,
 'parent_class': 'UserAttributes',
 'url': 'https://domo-community.domo.com/api/user/v1/properties/meta/keys/TestLibraryStore'}
get_data: sending json
get_data_response <Response [200 OK]>


UserAttribute(id='TestLibraryStore', name='TestLibraryStore', description='domo_libary upsert test 2024-01-03 - 04:51', issuer_type=<UserAttributes_IssuerType.CUSTOM: 'customer-defined'>, customer_id='mmmm-0012-0200', value_type='STRING', validator='ANY_VALUE', validator_configuration=None, security_voter='FULL_VIS_ADMIN_IDP', custom=True)

In [ ]:
# | exporti
@patch_to(UserAttributes)
async def delete(
    self: UserAttributes,
    attribute_id: str,
    session: httpx.AsyncClient = None,
    debug_api: bool = False,
    debug_num_stacks_to_drop=2,
    return_raw: bool = False,
):

    auth = self.auth

    res = await user_attribute_routes.delete_user_attribute(
        auth=auth,
        session=session,
        debug_api=debug_api,
        parent_class=self.__class__.__name__,
        debug_num_stacks_to_drop=debug_num_stacks_to_drop,
        attribute_id=attribute_id,
    )

    await self.get_attributes()

    return res

In [ ]:
show_doc(UserAttributes.delete)

---

[source](https://github.com/jaewilson07/domo_library/blob/main/domolibrary/classes/DomoInstanceConfig_UserAttribute.py#L282){target="_blank" style="float:right; font-size:smaller"}

### UserAttributes.delete

>      UserAttributes.delete (attribute_id:str, session:httpx.AsyncClient=None,
>                             debug_api:bool=False, debug_num_stacks_to_drop=2,
>                             return_raw:bool=False)

In [ ]:
# import os
# import domolibrary.client.DomoAuth as dmda

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

domo_user_attributes = UserAttributes(auth=token_auth)

await domo_user_attributes.delete(
    attribute_id="TestLibraryStore",
)

ResponseGetData(status=200, response='deleted TestLibraryStore', is_success=True, parent_class=None)

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()